## Imports

In [66]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [67]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 10th Feb. 17th


In [68]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [69]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [70]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [71]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [72]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-10-2025,Condo/Co-op,7200 NW 114th Ave #307,Doral,FL,33178.0,355000.0,2.0,2.0,PALM GARDENS AT DORAL CON,870.0,NaN,2003.0,NaN,408.0,342.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/7200-NW-114th-...,MARMLS,A11734388,N,Y,25.838392,-80.383112


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [73]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [74]:
df2 = df[(df['SOLD DATE'] > '2025-02-10') & (df['SOLD DATE'] < '2025-02-18')]

In [75]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-14    26
2025-02-13    20
2025-02-12    19
2025-02-11    11
2025-02-17     1
Name: count, dtype: int64

In [76]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [78]:
df_top_ten = df2.head(10)

In [79]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
30,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013
192,PAST SALE,2025-02-12,Condo/Co-op,16901 Collins Ave #3305,Sunny Isles Beach,FL,33160.0,5250000.0,4.0,5.5,JADE SIGNATURE CONDO,2937.0,NaN,2018.0,NaN,1788.0,6000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11652438,N,Y,25.932764,-80.121036
148,PAST SALE,2025-02-12,Condo/Co-op,400 Alton Rd #2406,Miami Beach,FL,33139.0,3900000.0,3.0,3.0,MURANO GRANDE AT PORTOFIN,2024.0,NaN,2003.0,NaN,1927.0,2849.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11665907,N,Y,25.772744,-80.140607
24,PAST SALE,2025-02-12,Condo/Co-op,10155 Collins Ave #1701,Bal Harbour,FL,33154.0,2850000.0,3.0,3.5,BAL HARBOUR 101 CONDO,3300.0,NaN,1978.0,NaN,864.0,4123.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,MARMLS,A11698460,N,Y,25.893258,-80.123056
242,PAST SALE,2025-02-14,Condo/Co-op,2301 Collins Ave #509,Miami Beach,FL,33139.0,2150000.0,2.0,2.0,RONEY PALACE CONDO,1480.0,NaN,1926.0,NaN,1453.0,2790.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11526922,N,Y,25.798440,-80.127580
72,PAST SALE,2025-02-11,Condo/Co-op,100 Lincoln Rd #1248,Miami Beach,FL,33139.0,1500100.0,2.0,2.0,THE DECOPLAGE CONDO,1190.0,NaN,1965.0,NaN,1261.0,1699.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-Linc...,MARMLS,A11702750,N,Y,25.790439,-80.129694
152,PAST SALE,2025-02-11,Condo/Co-op,300 Sunny Isles Blvd Unit 4-2301,Sunny Isles Beach,FL,33160.0,1415000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,748.0,1575.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11674751,N,Y,25.929383,-80.125611
164,PAST SALE,2025-02-13,Condo/Co-op,1800 Sunset Harbour Dr #1910,Miami Beach,FL,33139.0,1400000.0,2.0,2.0,SUNSET HARBOUR SOUTH COND,1120.0,NaN,1998.0,NaN,1250.0,1694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11668299,N,Y,25.794488,-80.144801
169,PAST SALE,2025-02-14,Condo/Co-op,2900 NE 7th Ave #902,Miami,FL,33137.0,1400000.0,3.0,4.0,BISCAYNE BEACH CONDO,1698.0,NaN,2017.0,NaN,824.0,2419.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11664996,N,Y,25.805017,-80.185768
225,PAST SALE,2025-02-14,Condo/Co-op,17121 Collins Ave #1607,Sunny Isles Beach,FL,33160.0,1300000.0,1.0,2.0,JADE OCEAN CONDO,1161.0,NaN,2009.0,NaN,1120.0,1857.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11606593,N,Y,25.934401,-80.121005


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [80]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [81]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [82]:
response_df = pd.DataFrame(response_list)

In [83]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [84]:
df_top_ten = merged_df

In [85]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None
1,PAST SALE,2025-02-12,Condo/Co-op,16901 Collins Ave #3305,Sunny Isles Beach,FL,33160.0,5250000.0,4.0,5.5,JADE SIGNATURE CONDO,2937.0,NaN,2018.0,NaN,1788.0,6000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11652438,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Alex De Leon,Fortune Christie's International Real Estate,None,None,Pamela DiMartino,Coldwell Banker Realty,Daniel Snyder,"Compass Florida, LLC."
2,PAST SALE,2025-02-12,Condo/Co-op,400 Alton Rd #2406,Miami Beach,FL,33139.0,3900000.0,3.0,3.0,MURANO GRANDE AT PORTOFIN,2024.0,NaN,2003.0,NaN,1927.0,2849.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11665907,N,Y,25.772744,-80.140607,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Jeff Miller,One Sotheby's International Re,None,None,Dana Munick,"Income Real Estate, Inc.",None,None
3,PAST SALE,2025-02-12,Condo/Co-op,10155 Collins Ave #1701,Bal Harbour,FL,33154.0,2850000.0,3.0,3.5,BAL HARBOUR 101 CONDO,3300.0,NaN,1978.0,NaN,864.0,4123.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,MARMLS,A11698460,N,Y,25.893258,-80.123056,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,Tara Elias Schuchts,Douglas Elliman,None,None,Teo Palmieri,Elite Ocean View Realty LLC,None,None
4,PAST SALE,2025-02-14,Condo/Co-op,2301 Collins Ave #509,Miami Beach,FL,33139.0,2150000.0,2.0,2.0,RONEY PALACE CONDO,1480.0,NaN,1926.0,NaN,1453.0,2790.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11526922,N,Y,25.798440,-80.127580,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Danell Van Orden,The Keyes Company,None,None,Michael Guay,Alure Capital,None,None
5,PAST SALE,2025-02-11,Condo/Co-op,100 Lincoln Rd #1248,Miami Beach,FL,33139.0,1500100.0,2.0,2.0,THE DECOPLAGE CONDO,1190.0,NaN,1965.0,NaN,1261.0,1699.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-Linc...,MARMLS,A11702750,N,Y,25.790439,-80.129694,https://www.redfin.com/FL/Miami-Beach/100-Linc...,Patricia Schille Hubalek,Miami Dreams Realty Inc.,None,None,Sophie Albrecht,Thine Agency,None,None
6,PAST SALE,2025-02-11,Condo/Co-op,300 Sunny Isles Blvd Unit 4-2301,Sunny Isles Beach,FL,33160.0,1415000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,748.0,1575.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11674751,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Netasway Hirsch,The Agency Realty Corp,Maria Luz,The Agency Realty Corp,Ekaterina Lavander,Beachfront Realty Inc,None,None
7,PAST SALE,2025-02-13,Condo/Co-op,1800 Sunset Harbour Dr #1910,Miami Beach,FL,33139.0,1400000.0,2.0,2.0,SUNSET HARBOUR SOUTH COND,1120.0,NaN,1998.0,NaN,1250.0,1694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11668299,N,Y,25.794488,-80.144801,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,Paul Sasseville,"Compass Florida, LLC.",None,None,Paul Sasseville,"Compass Florida, LLC.",None,None
8,PAST SALE,2025-02-14,Condo/Co-op,2900 NE 7th Ave #902,Miami,FL,33137.0,1400000.0,3.0,4.0,BISCAYNE BEACH CONDO,1698.0,NaN,2017.0,NaN,824.0,2419.0,Sold,NaN,NaN,https://www.redfin.co

## Current Week's Values

In [86]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

77


In [87]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$924,040


In [88]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$557


In [89]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$71,151,100


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [31]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
84
Input Previous Week Condo Average Sales Price:
805279
Input Previous Week Condo Average PSF:
567
Input Previous Week Condo Sales total (ex: 198_000_000)
67_600_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [90]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [91]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [92]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [93]:
pd.set_option('display.max_columns',None)

In [94]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None,1,orange


In [95]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [96]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-02-12,Condo/Co-op,16901 Collins Ave #3305,Sunny Isles Beach,FL,33160.0,5250000.0,4.0,5.5,JADE SIGNATURE CONDO,2937.0,NaN,2018.0,NaN,1788.0,6000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11652438,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Alex De Leon,Fortune Christie's International Real Estate,None,None,Pamela DiMartino,Coldwell Banker Realty,Daniel Snyder,"Compass Florida, LLC.",2,blue
2,PAST SALE,2025-02-12,Condo/Co-op,400 Alton Rd #2406,Miami Beach,FL,33139.0,3900000.0,3.0,3.0,MURANO GRANDE AT PORTOFIN,2024.0,NaN,2003.0,NaN,1927.0,2849.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11665907,N,Y,25.772744,-80.140607,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Jeff Miller,One Sotheby's International Re,None,None,Dana Munick,"Income Real Estate, Inc.",None,None,3,blue
3,PAST SALE,2025-02-12,Condo/Co-op,10155 Collins Ave #1701,Bal Harbour,FL,33154.0,2850000.0,3.0,3.5,BAL HARBOUR 101 CONDO,3300.0,NaN,1978.0,NaN,864.0,4123.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,MARMLS,A11698460,N,Y,25.893258,-80.123056,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,Tara Elias Schuchts,Douglas Elliman,None,None,Teo Palmieri,Elite Ocean View Realty LLC,None,None,4,blue
4,PAST SALE,2025-02-14,Condo/Co-op,2301 Collins Ave #509,Miami Beach,FL,33139.0,2150000.0,2.0,2.0,RONEY PALACE CONDO,1480.0,NaN,1926.0,NaN,1453.0,2790.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11526922,N,Y,25.798440,-80.127580,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Danell Van Orden,The Keyes Company,None,None,Michael Guay,Alure Capital,None,None,5,blue
5,PAST SALE,2025-02-11,Condo/Co-op,100 Lincoln Rd #1248,Miami Beach,FL,33139.0,1500100.0,2.0,2.0,THE DECOPLAGE CONDO,1190.0,NaN,1965.0,NaN,1261.0,1699.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-Linc...,MARMLS,A11702750,N,Y,25.790439,-80.129694,https://www.redfin.com/FL/Miami-Beach/100-Linc...,Patricia Schille Hubalek,Miami Dreams Realty Inc.,None,None,Sophie Albrecht,Thine Agency,None,None,6,blue
6,PAST SALE,2025-02-11,Condo/Co-op,300 Sunny Isles Blvd Unit 4-2301,Sunny Isles Beach,FL,33160.0,1415000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,748.0,1575.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11674751,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Netasway Hirsch,The Agency Realty Corp,Maria Luz,The Agency Realty Corp,Ekaterina Lavander,Beachfront Realty Inc,None,None,7,blue
7,PAST SALE,2025-02-13,Condo/Co-op,1800 Sunset Harbour Dr #1910,Miami Beach,FL,33139.0,1400000.0,2.0,2.0,SUNSET HARBOUR SOUTH COND,1120.0,NaN,1998.0,NaN,1250.0,1694.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11668299,N,Y,25.794488,-80.144801,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,Paul Sasseville,"Compass Florida, LLC.",None,None,Paul Sasseville,"Compass Florida, LLC.",None,None,8,blue
8,PAST SALE,2025-02-14,Condo/Co-op,2900 NE 7th Ave #902,Miami,FL,33137.0,1400000.0,3.0,4.0,BISCAYNE BEACH CONDO,

In [97]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [98]:
m.save('index.html')

## Data snagger

In [99]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [100]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [101]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Miami-Dade_County_condo_sales_week_ending_02182024


# CREATE TEMPLATE 

In [102]:
muni_set = set(df_top_ten['CITY'])

In [103]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [104]:
df_top_ten.reset_index(inplace=True,drop=True)

In [105]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [106]:
top_sale

'C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit PHA in Miami'

In [107]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [108]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [109]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None,1,orange,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
1,PAST SALE,2025-02-12,Condo/Co-op,16901 Collins Ave #3305,Sunny Isles Beach,FL,33160.0,5250000.0,4.0,5.5,JADE SIGNATURE CONDO,2937.0,NaN,2018.0,NaN,1788.0,6000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11652438,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Alex De Leon,Fortune Christie's International Real Estate,None,None,Pamela DiMartino,Coldwell Banker Realty,Daniel Snyder,"Compass Florida, LLC.",2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #330...
2,PAST SALE,2025-02-12,Condo/Co-op,400 Alton Rd #2406,Miami Beach,FL,33139.0,3900000.0,3.0,3.0,MURANO GRANDE AT PORTOFIN,2024.0,NaN,2003.0,NaN,1927.0,2849.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11665907,N,Y,25.772744,-80.140607,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Jeff Miller,One Sotheby's International Re,None,None,Dana Munick,"Income Real Estate, Inc.",None,None,3,blue,MURANO GRANDE AT PORTOFIN at 400 Alton Rd #240...
3,PAST SALE,2025-02-12,Condo/Co-op,10155 Collins Ave #1701,Bal Harbour,FL,33154.0,2850000.0,3.0,3.5,BAL HARBOUR 101 CONDO,3300.0,NaN,1978.0,NaN,864.0,4123.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,MARMLS,A11698460,N,Y,25.893258,-80.123056,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,Tara Elias Schuchts,Douglas Elliman,None,None,Teo Palmieri,Elite Ocean View Realty LLC,None,None,4,blue,BAL HARBOUR 101 CONDO at 10155 Collins Ave #17...
4,PAST SALE,2025-02-14,Condo/Co-op,2301 Collins Ave #509,Miami Beach,FL,33139.0,2150000.0,2.0,2.0,RONEY PALACE CONDO,1480.0,NaN,1926.0,NaN,1453.0,2790.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11526922,N,Y,25.798440,-80.127580,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Danell Van Orden,The Keyes Company,None,None,Michael Guay,Alure Capital,None,None,5,blue,RONEY PALACE CONDO at 2301 Collins Ave #509 in...
5,PAST SALE,2025-02-11,Condo/Co-op,100 Lincoln Rd #1248,Miami Beach,FL,33139.0,1500100.0,2.0,2.0,THE DECOPLAGE CONDO,1190.0,NaN,1965.0,NaN,1261.0,1699.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-Linc...,MARMLS,A11702750,N,Y,25.790439,-80.129694,https://www.redfin.com/FL/Miami-Beach/100-Linc...,Patricia Schille Hubalek,Miami Dreams Realty Inc.,None,None,Sophie Albrecht,Thine Agency,None,None,6,blue,THE DECOPLAGE CONDO at 100 Lincoln Rd #1248 in...
6,PAST SALE,2025-02-11,Condo/Co-op,300 Sunny Isles Blvd Unit 4-2301,Sunny Isles Beach,FL,33160.0,1415000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,748.0,1575.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11674751,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Netasway Hirsch,The Agency Realty Corp,Maria Luz,The Agency Realty Corp,Ekaterina Lavander,Beachfront Realty Inc,None,None,7,blue,PARQUE TOWERS CONDO at 300 Sunny Isles Blvd Un...
7,PAST SALE,2025-02-13,Condo/Co-op,1800 Sunset Harbour Dr #1910,Miami Beach,FL,33139.0,1400000.0,2.0,2.0,SUNSET HARBOUR SOUTH COND,1120.0,NaN,1998.0,NaN,1250.0,1694.0,Sold,NaN,NaN,h

In [110]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: C.G.B. SUBDIVISION closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,300,000 to $22,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Bal Harbour, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 77 condo sales totaling $71,151,100 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 84 condo sales totaling $67,600,000.

Last week’s units sold for an average of $924,040, higher than the $805,279 average price from the previous week. The avera

In [111]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [112]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [113]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [115]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: C.G.B. SUBDIVISION closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,300,000 to $22,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Bal Harbour, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 77 condo sales totaling $71,151,100 million from Feb. 10th to Feb. 17th. The previous week, brokers closed 84 condo sales totaling $67,600,000.

Last week’s units sold for an average of $924,040, higher than the $805,279 average price from the previous week. The avera

In [116]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami/2811-S-Bayshore-Dr-33133/unit-PHA/home/193630653


In [117]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-3305/home/56743883


In [118]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17121-Collins-Ave-33160/unit-1607/home/43416046


In [119]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/2900-NE-7th-Ave-33137/unit-902/home/79132237


In [120]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/2811-S-Bayshore-Dr-33133/unit-PHA/home/193630653


In [121]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/300-Sunny-Isles-Blvd-33160/unit-4-2301/home/56744533


In [122]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-14,Condo/Co-op,2811 S Bayshore Dr Unit PHA,Miami,FL,33133.0,22500000.0,5.0,6.5,C.G.B. SUBDIVISION,6421.0,NaN,2020.0,NaN,3504.0,12406.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11678713,N,Y,25.728751,-80.238013,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,Judith Zeder,Coldwell Banker Realty,Nathan Zeder,Coldwell Banker Realty,Julian Cohen,Douglas Elliman,None,None,1,orange,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
1,PAST SALE,2025-02-12,Condo/Co-op,16901 Collins Ave #3305,Sunny Isles Beach,FL,33160.0,5250000.0,4.0,5.5,JADE SIGNATURE CONDO,2937.0,NaN,2018.0,NaN,1788.0,6000.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11652438,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Alex De Leon,Fortune Christie's International Real Estate,None,None,Pamela DiMartino,Coldwell Banker Realty,Daniel Snyder,"Compass Florida, LLC.",2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #330...
2,PAST SALE,2025-02-12,Condo/Co-op,400 Alton Rd #2406,Miami Beach,FL,33139.0,3900000.0,3.0,3.0,MURANO GRANDE AT PORTOFIN,2024.0,NaN,2003.0,NaN,1927.0,2849.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11665907,N,Y,25.772744,-80.140607,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Jeff Miller,One Sotheby's International Re,None,None,Dana Munick,"Income Real Estate, Inc.",None,None,3,blue,MURANO GRANDE AT PORTOFIN at 400 Alton Rd #240...
3,PAST SALE,2025-02-12,Condo/Co-op,10155 Collins Ave #1701,Bal Harbour,FL,33154.0,2850000.0,3.0,3.5,BAL HARBOUR 101 CONDO,3300.0,NaN,1978.0,NaN,864.0,4123.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,MARMLS,A11698460,N,Y,25.893258,-80.123056,https://www.redfin.com/FL/Bal-Harbour/10155-Co...,Tara Elias Schuchts,Douglas Elliman,None,None,Teo Palmieri,Elite Ocean View Realty LLC,None,None,4,blue,BAL HARBOUR 101 CONDO at 10155 Collins Ave #17...
4,PAST SALE,2025-02-14,Condo/Co-op,2301 Collins Ave #509,Miami Beach,FL,33139.0,2150000.0,2.0,2.0,RONEY PALACE CONDO,1480.0,NaN,1926.0,NaN,1453.0,2790.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11526922,N,Y,25.798440,-80.127580,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Danell Van Orden,The Keyes Company,None,None,Michael Guay,Alure Capital,None,None,5,blue,RONEY PALACE CONDO at 2301 Collins Ave #509 in...
5,PAST SALE,2025-02-11,Condo/Co-op,100 Lincoln Rd #1248,Miami Beach,FL,33139.0,1500100.0,2.0,2.0,THE DECOPLAGE CONDO,1190.0,NaN,1965.0,NaN,1261.0,1699.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-Linc...,MARMLS,A11702750,N,Y,25.790439,-80.129694,https://www.redfin.com/FL/Miami-Beach/100-Linc...,Patricia Schille Hubalek,Miami Dreams Realty Inc.,None,None,Sophie Albrecht,Thine Agency,None,None,6,blue,THE DECOPLAGE CONDO at 100 Lincoln Rd #1248 in...
6,PAST SALE,2025-02-11,Condo/Co-op,300 Sunny Isles Blvd Unit 4-2301,Sunny Isles Beach,FL,33160.0,1415000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,748.0,1575.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11674751,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Netasway Hirsch,The Agency Realty Corp,Maria Luz,The Agency Realty Corp,Ekaterina Lavander,Beachfront Realty Inc,None,None,7,blue,PARQUE TOWERS CONDO at 300 Sunny Isles Blvd Un...
7,PAST SALE,2025-02-13,Condo/Co-op,1800 Sunset Harbour Dr #1910,Miami Beach,FL,33139.0,1400000.0,2.0,2.0,SUNSET HARBOUR SOUTH COND,1120.0,NaN,1998.0,NaN,1250.0,1694.0,Sold,NaN,NaN,h

## Time on Market Calculator

In [124]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 1) ## List (Earlier) date
date2 = datetime(2025, 2, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

164


## Clean CSV for Datawrapper Chart

In [125]:
chart_df = df_top_ten

In [126]:
chart_df = chart_df.fillna(" ")

In [127]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [128]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [129]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [130]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [131]:
chart_df['$/SQUARE FEET'].astype(float)

0    3504.0
1    1788.0
2    1927.0
3     864.0
4    1453.0
5    1261.0
6     748.0
7    1250.0
8     824.0
9    1120.0
Name: $/SQUARE FEET, dtype: float64

In [132]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [133]:
csv_date_string = today.strftime("%m_%d_%Y")

In [134]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [135]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-14,2811 S Bayshore Dr Unit PHA in Miami,"$22,500,000",5,6.5,"6,421",2020,"$3,504",MARMLS,A11678713,Judith Zeder Coldwell Banker Realty Nathan Zed...,Julian Cohen Douglas Elliman
1,2025-02-12,16901 Collins Ave #3305 in Sunny Isles Beach,"$5,250,000",4,5.5,"2,937",2018,"$1,788",MARMLS,A11652438,Alex De Leon Fortune Christie's International ...,Pamela DiMartino Coldwell Banker Realty Daniel...
2,2025-02-12,400 Alton Rd #2406 in Miami Beach,"$3,900,000",3,3.0,"2,024",2003,"$1,927",MARMLS,A11665907,Jeff Miller One Sotheby's International Re,"Dana Munick Income Real Estate, Inc."
3,2025-02-12,10155 Collins Ave #1701 in Bal Harbour,"$2,850,000",3,3.5,"3,300",1978,$864,MARMLS,A11698460,Tara Elias Schuchts Douglas Elliman,Teo Palmieri Elite Ocean View Realty LLC
4,2025-02-14,2301 Collins Ave #509 in Miami Beach,"$2,150,000",2,2.0,"1,480",1926,"$1,453",MARMLS,A11526922,Danell Van Orden The Keyes Company,Michael Guay Alure Capital
5,2025-02-11,100 Lincoln Rd #1248 in Miami Beach,"$1,500,100",2,2.0,"1,190",1965,"$1,261",MARMLS,A11702750,Patricia Schille Hubalek Miami Dreams Realty I...,Sophie Albrecht Thine Agency
6,2025-02-11,300 Sunny Isles Blvd Unit 4-2301 in Sunny Isle...,"$1,415,000",3,3.5,"1,892",2019,$748,MARMLS,A11674751,Netasway Hirsch The Agency Realty Corp Maria L...,Ekaterina Lavander Beachfront Realty Inc
7,2025-02-13,1800 Sunset Harbour Dr #1910 in Miami Beach,"$1,400,000",2,2.0,"1,120",1998,"$1,250",MARMLS,A11668299,"Paul Sasseville Compass Florida, LLC.","Paul Sasseville Compass Florida, LLC."
8,2025-02-14,2900 NE 7th Ave #902 in Miami,"$1,400,000",3,4.0,"1,698",2017,$824,MARMLS,A11664996,Alex Fernandez-Pascual Fortune Christie's Inte...,Alex Fernandez-Pascual Fortune Christie's Inte...
9,2025-02-14,17121 Collins Ave #1607 in Sunny Isles Beach,"$1,300,000",1,2.0,"1,161",2009,"$1,120",MARMLS,A11606593,Maria Carrillo The Keyes Company,Laura Rojas Gechidjian Beachfront Realty Inc
